In [3]:
import pandas as pd
import glob
from datetime import datetime
import os
from sqlalchemy import create_engine
import psycopg2
import json
import numpy as np
import sqlalchemy as sa


In [4]:
password = os.environ['ML_POSTGRESS_URL'].split(':')[2].split("@")[0]
host = os.environ['ML_POSTGRESS_HOST']
database = "simulation"
engine = sa.create_engine( f"postgresql://max:{password}@{host}" + f"/{database}")
conn = psycopg2.connect(host=host, dbname=database, user='max', password=password)
sql_query = f"""
SELECT 
    *
FROM 
    mlr_simulations_dictionary;
"""
df= pd.read_sql(sql_query, conn)
df

/var/folders/h2/7j0s980d2w7951jcy8zc25ww08y435/T/ipykernel_37652/2821676896.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(sql_query, conn)


,simulation_id,name,seed,n_classes,n_features,n_samples,n_informative,flip_y,base_rates,coef_scale,class_sep,notes
0,2619350859,"mlr_3_class_10_842_1000__0.6_0.02_1.4_0.35,0.3...",42.0,3,10,1000,8,0.020,"[0.35, 0.35, 0.3]",0.6,1.4,
1,2562140346,"mlr_3_class_50_4542_1000__0.6_0.02_1.4_0.35,0....",42.0,3,50,1000,45,0.020,"[0.35, 0.35, 0.3]",0.6,1.4,
2,4262490533,"mlr_3_class_10_842_1000__0.8_0.02_1.4_0.35,0.3...",42.0,3,10,1000,8,0.020,"[0.35, 0.35, 0.3]",0.8,1.4,
3,1695645375,"mlr_3_class_10_842_1000__0.8_0.02_1.3_0.35,0.3...",42.0,3,10,1000,8,0.020,"[0.35, 0.35, 0.3]",0.8,1.3,
4,791672308,"mlr_3_class_10_842_1000__0.8_0.05_1.3_0.35,0.3...",42.0,3,10,1000,8,0.050,"[0.35, 0.35, 0.3]",0.8,1.3,
...,...,...,...,...,...,...,...,...,...,...,...,...
61,2124861435,"mlr_6_class_8_842_1000__0.8_0_1.9_0.2,0.2,0.15...",42.0,6,8,1000,8,0.000,"[0.2, 0.2, 0.15, 0.15, 0.15, 0.15]",0.8,1.9,
62,3811716761,"mlr_6_class_50_2542_1000__0.8_0_1.9_0.2,0.2,0....",42.0,6,50,1000,25,0.000,"[0.2, 0.2, 0.15, 0.15, 0.15, 0.15]",0.8,1.9,
63,729968159,"mlr_6_class_50_2542_1000__0.8_0.01_1.9_0.2,0.2...",42.0,6,50,1000,25,0.010,"[0.2, 0.2, 0.15, 0.15, 0.15, 0.15]",0.8,1.9,
64,765945077,"mlr_6_class_50_2542_1000__0.8_0.001_1.9_0.2,0....",42.0,6,50,1000,25,0.001,"[0.2, 0.2, 0.15, 0.15, 0.15, 0.15]",0.8,1.9,


In [10]:
from generate_data import simulate_mnlogit
for index, row in df.iterrows():
    dataset_name = row['name']
    simulation_id = row['simulation_id']

    seed = int(float(row['seed']))
    n_samples = row['n_samples']
    n_classes = row['n_classes']
    n_features = row['n_features']
    base_rates = json.loads(row['base_rates'])
    coef_scale = row['coef_scale']
    class_sep = row['class_sep']
    n_informative = row['n_informative']
    flip_y = float(row['flip_y'])
    df1, params = simulate_mnlogit(
        seed=seed,
        n_samples=n_samples, n_classes=n_classes, n_features=n_features,
        base_rates=base_rates,
        coef_scale=coef_scale, class_sep=class_sep,
        n_informative=n_informative, flip_y=flip_y,
        return_probas=False
    )
    print(df1.head())


         X0        X1        X2        X3        X4        X5        X6  \
0  0.304717 -1.039984  0.750451  0.940565 -1.951035 -1.302180  0.127840   
1  0.879398  0.777792  0.066031  1.127241  0.467509 -0.859292  0.368751   
2 -0.184862 -0.680930  1.222541 -0.154529 -0.428328 -0.352134  0.532309   
3  2.141648 -0.406415 -0.512243 -0.813773  0.615979  1.128972 -0.113947   
4  0.743254  0.543154 -0.665510  0.232161  0.116686  0.218689  0.871429   

         X7        X8        X9  Y  
0 -0.316243 -0.016801 -0.853044  1  
1 -0.958883  0.878450 -0.049926  2  
2  0.365444  0.412733  0.430821  1  
3 -0.840156 -0.824481  0.650593  2  
4  0.223596  0.678914  0.067579  2  
         X0        X1        X2        X3        X4        X5        X6  \
0  0.304717 -1.039984  0.750451  0.940565 -1.951035 -1.302180  0.127840   
1  0.289119  0.631288 -1.457156 -0.319671 -0.470373 -0.638878 -0.275142   
2 -0.378163  1.299228 -0.356264  0.737516 -0.933618 -0.205438 -0.950022   
3 -0.158635 -1.035654 -1.67

/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: divide by zero encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: overflow encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: invalid value encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: divide by zero encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: overflow encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: invalid value encountered in matmul
  P = softmax(X @ W + b, axis=1)
/Users/22701987/Documents/code/simulation/generate_data.py:49: RuntimeWarning: divide by zero encountered in matmul
  P 